In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
#import seaborn
#seaborn.set_context(context="talk")
%matplotlib inline

참고문헌  
\[1\] [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html#prelims)  
\[2\] [new The Annotated Transformer](https://nlp.seas.harvard.edu/annotated-transformer/)  
\[3\] [git for new The Annotated Transforme](https://github.com/harvardnlp/annotated-transformer.git)

In [2]:
def create_padding_mask(x,pad=0):
    """
    x: (batch_size, seq_len)
    return: mask of shape (batch_size, 1, 1, seq_len)
    """
    mask = (x != pad).float()
    return mask.unsqueeze(1).unsqueeze(1)

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
        
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

### Fig1. Left

![fig_1](../images/add_norm.png)

We employ a residual connection ([Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)) around each of the two sub-layers,   
followed by layer normalization ([Layer Normalization](https://arxiv.org/abs/1607.06450)).

In [3]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

여기서, self.a_2 * x 연산은  원소별 곱셉을 의미함.

In [4]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

``` python
 x + self.dropout(sublayer(self.norm(x)))
```

이 방식은 Pre-Layer Normalization 또는 Pre-Norm 방식임.  
이 방식은 Transformer의 초기 버전과는 반대되는 구조이며, 여러 현대 Transformer 변형  
모델에서 선호됨. (gpt에게 물어봄)

왜 Pre-Norm을 사용하나?  
1. 학습 안정성 향상  
깊은 Transformer 구조에서는 Post-Norm(LayerNorm(x + Sublayer(x))) 구조가  
그래디언트 소실 문제를 야기할 수 있습니다.  
반면, Pre-Norm은 정규화를 먼저 적용해 잔차 경로를 더 잘 유지할 수 있어서  
깊은 네트워크에서도 학습이 안정적으로 진행됩니다.

2. 그래디언트 흐름 개선  
Pre-Norm은 x 경로가 LayerNorm 앞에 있기 때문에, 그래디언트가 직접 x로 잘 전달됩니다.  
즉, 그래디언트가 서브레이어(sublayer)에서 멈추지 않고 잘 흘러서, 깊은 모델 학습에 유리합니다.  

3. 최근 모델들이 채택  
GPT-2, T5, BERT-large 등 다양한 Transformer 기반 모델들이 이 구조 또는 변형 구조를 사용.  
특히 T5는 모든 블록에 Pre-Norm을 적용하여 안정성과 효율성을 확보합니다.


4. [Understanding the Difficulty of Training Transformers](https://arxiv.org/abs/2004.08249) 논문에서 Pre-Norm 구조의 이점이 설명됨. 

즉, 각 서브레이어(sub-layer)의 출력은 $\text{LayerNorm}(x + \text{Sublayer}(x))$  
의 형태입니다. 여기서 $\text{Sublayer}(x)$ 는 해당 서브레이어가 구현하는 함수입니다.  
우리는 각 서브레이어의 출력에 **드롭아웃(dropout)**을 적용한 후, 이를 서브레이어의 입력에  
더하고 정규화를 수행합니다 ([Dropout](https://jmlr.org/papers/v15/srivastava14a.html)).

이러한 **잔차 연결(residual connections)** 을 원활하게 하기 위해, 모델 내의   
모든 서브레이어들과 임베딩 레이어는 출력 차원을 $d_{model} = 512$로 맞춥니다.

EncoderLayer에서 $size=d_{model}=512$를 의미함.

In [5]:

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

각 레이어는 두 개의 서브레이어로 구성되어 있습니다.  
첫 번째는 멀티-헤드 자기-어텐션(Multi-head Self-Attention) 메커니즘이며,  
두 번째는 위치별로 독립적으로 작동하는 완전 연결 피드포워드 네트워크  
(Position-wise Fully Connected Feed-Forward Network)입니다.

``` python
x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
return self.sublayer[1](x, self.feed_forward)
```
위 코드를 그림으로 표현하면  
![](../images/add_norm_connections.png)

The encoder is composed of a stack of $N=6$ identical layers.

In [6]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

encoder 출력 마지막에 LayerNorm을 수행하는 것은 논문에 언급되지 않았지만,  
논문 저자들의 원래 구현(예: [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor)) 및 후속 PyTorch 구현체들에서는  
encoder 마지막에 추가적인 LayerNorm을 수행하는 것이 관행이 되었다고 함.

### hyper-parameters

In [7]:
N=6
d_model=512
d_ff=2048
h=8
dropout=0.1

하이퍼 파라미터를 위의 경우처럼 설정해주면,  
각 layer의 입력 출력 데이터의 차원은 (n_batch, seq_len, d_model) 차원이 됨.  
여기서 seq_len는 입력 데이터의 문장 길이임.  
문장이 "nice to meet you" 면, seg_len은 5가 되면,  
하지만 학습을 위해 문장의 길이는 통일하는 것으로 보임. 그래서 pad가 필요하고,  
mask가 필요함.

In [8]:
embedding = Embeddings(d_model,11)

In [9]:
c = copy.deepcopy
attn = MultiHeadedAttention(h, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
position = PositionalEncoding(d_model, dropout)

In [10]:
encoder_layer = EncoderLayer(d_model, c(attn), c(ff), dropout)

In [11]:
encoder = Encoder(encoder_layer, N);

In [12]:
x = torch.from_numpy(np.random.randint(0, 11, size=(1, 10)))
xx = embedding(x)
print("xx input shape : ", xx.shape)
pad_mask = create_padding_mask(x)

xx input shape :  torch.Size([1, 10, 512])


In [14]:
x

tensor([[8, 7, 0, 2, 7, 7, 2, 1, 7, 4]])

In [13]:
y = encoder(xx, pad_mask)
y.shape

torch.Size([1, 10, 512])